In [1]:
import os
import pandas
import warnings
warnings.filterwarnings("ignore")

datadir = '/home/idies/workspace/raddick_census/pop2016'
os.chdir(datadir)

os.getcwd()

'/home/idies/workspace/raddick_census/pop2016'

In [17]:
counties = pandas.read_csv('co-est2016-alldata.csv', encoding='utf-8')
counties = counties[counties['SUMLEV'] == 50]
counties['GEOID'] = counties['SUMLEV'].apply(lambda x: '{:03d}'.format(x)) + '00US' +  counties['STATE'].apply(lambda x: '{:02d}'.format(x)) + counties['COUNTY'].apply(lambda x: '{:03d}'.format(x)) 
counties = counties.set_index('GEOID')

counties_out = pandas.DataFrame()
counties_out = counties_out.append(counties[(counties['STATE'] == 6) & (counties['COUNTY'].apply(lambda x: x in [37, 71, 59]))])
counties_out = counties_out.append(counties[(counties['STATE'] == 34) & (counties['COUNTY'] == 5)])
counties_out = counties_out.append(counties[(counties['STATE'] == 36) & (counties['COUNTY'] == 119)])
counties_out = counties_out.append(counties[(counties['STATE'] == 17) & (counties['COUNTY'].apply(lambda x: x in [31, 43, 89, 93, 197]))])
counties = counties.drop(counties_out.index, axis=0)


subdivs = pandas.read_csv('sub-est2016_all.csv', encoding='utf-8')

# California (state 6): Los Angeles (37), Orange (59), San Bernardino (71)
# New Jersey (state 34): Burlington (5)
# New York (state 36): Westchester (19)
# Illinois (state 17): Cook (31), DuPage (43), Kane (89), Kendall (93), Will (197)

ca = subdivs[(subdivs['PRIMGEO_FLAG'] == 1) & (subdivs['STATE'] == 6) & (subdivs['COUNTY'].apply(lambda x: x in [37, 59, 71]))]
ca['GEOID'] = ''

ca['GEOID'] = ca['SUMLEV'].apply(lambda x: '{:03d}'.format(x)).astype('str') + '06US' + ca['PLACE'].apply(lambda x: '{:05d}'.format(x)).astype('str')

ca.loc[ca['PLACE'] == 99990, 'GEOID'] = ca['SUMLEV'].apply(lambda x: '{:03d}'.format(x)).astype('str') + '00US' + ca['STATE'].apply(lambda x: '{:02d}'.format(x)).astype('str') + ca['COUNTY'].apply(lambda x: '{:02d}'.format(x)).astype('str') + 'x'
#ca['GEOID'] = ca['SUMLEV'].apply(lambda x: '{:03d}'.format(x)).astype('str') + '6' + ca['COUNTY'].apply(lambda x: '{:02d}'.format(x)).astype('str') + 'U' + ca['PLACE'].apply(lambda x: '{:05d}'.format(x)).astype('str')

#ca.loc[(ca['SUMLEV'] == 157) & (ca['FUNCSTAT'] == 'F'), 'GEOID'] = ca['SUMLEV'].apply(lambda x: '{:03d}'.format(x)) + '00US' + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + ca['COUNTY'].apply(lambda x: '{:03d}'.format(x)) + 'xx'
ca = ca.set_index('GEOID')

nj = subdivs[(subdivs['PRIMGEO_FLAG'] == 1) & (subdivs['STATE'] == 34) & (subdivs['COUNTY'].apply(lambda x: x in [5]))]
nj['GEOID'] = nj['SUMLEV'].apply(lambda x: '{:02d}'.format(x)).astype('str') + '345US' + nj['COUSUB'].apply(lambda x: '{:05d}'.format(x)).astype('str')
nj = nj.set_index('GEOID')

ny = subdivs[(subdivs['PRIMGEO_FLAG'] == 1) & (subdivs['STATE'] == 36) & (subdivs['COUNTY'].apply(lambda x: x in [119]))]
ny['GEOID'] = ny['SUMLEV'].apply(lambda x: '{:02d}'.format(x)).astype('str') + '36119' + ny['COUSUB'].apply(lambda x: '{:05d}'.format(x)).astype('str')
ny = ny.set_index('GEOID')

il = subdivs[(subdivs['PRIMGEO_FLAG'] == 1) & (subdivs['STATE'] == 17) & (subdivs['COUNTY'].apply(lambda x: x in [31, 43, 89, 93, 197]))]

il['GEOID'] = il['SUMLEV'].apply(lambda x: '{:03d}'.format(x)).astype('str') + '17PL' + il['PLACE'].apply(lambda x: '{:05d}'.format(x)).astype('str')

il.loc[(il['PLACE'] == 99990), 'GEOID'] = il['SUMLEV'].apply(lambda x: '{:03d}'.format(x)).astype('str') + '17CS' + il['COUSUB'].apply(lambda x: '{:05d}'.format(x)).astype('str')
il.loc[(il['SUMLEV'] == 61), 'GEOID'] = il['SUMLEV'].apply(lambda x: '{:03d}'.format(x)).astype('str') + '17CS' + il['COUSUB'].apply(lambda x: '{:05d}'.format(x)).astype('str')
il = il.set_index('GEOID')


df = pandas.DataFrame()
df = df.append(counties)
df = df.append(ca)
df = df.append(nj)
df = df.append(ny)
df = df.append(il)

#df.sample(20)
show_columns = ['SUMLEV', 'STATE', 'COUNTY', 'STNAME']
show_columns += ['CENSUS2010POP', 'ESTIMATESBASE2010', 'POPESTIMATE2010']
show_columns += ['POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013']
show_columns += ['POPESTIMATE2014', 'POPESTIMATE2015', 'POPESTIMATE2016']
df[show_columns].head(1)

,SUMLEV,STATE,COUNTY,STNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016
GEOID,,,,,,,,,,,,,
05000US01001,50,1,1,Alabama,54571,54571,54742,55255,55027,54792,54977,55035,55416


In [33]:
new = pandas.read_csv('newstates.csv', dtype='str', low_memory=False)
new.dtypes
#new = new.set_index('GEOID')

#df[show_columns]
#df['POPESTIMATE2016'].sum()

#df = df.join(new, how='left', rsuffix='yyy')
#df[show_columns].head(1)
#df.groupby('state_new')['POPESTIMATE2016'].sum()

GEOID        object
state_new    object
dtype: object

In [ ]:
#counties = counties.set_index('GEOID')

# California (state 6): Los Angeles (37), Orange (59), San Bernardino (71)
# Illinois (state 17): Cook (31), DuPage (43), Kane (89), Kendall (93), Will (197)
# New Jersey (state 34): Burlington (5)
# New York (state 36): Westchester (19)

subdivs = pandas.DataFrame()
ca = pop2016_df[(pop2016_df['PRIMGEO_FLAG'] == 1) & (pop2016_df['STATE'] == 6) & (pop2016_df['COUNTY'].apply(lambda x: x in [37, 59, 71]))]
il = pop2016_df[(pop2016_df['PRIMGEO_FLAG'] == 1) & (pop2016_df['STATE'] == 17) & (pop2016_df['COUNTY'].apply(lambda x: x in [31, 43, 89, 93, 197]))]
nj = pop2016_df[(pop2016_df['PRIMGEO_FLAG'] == 1) & (pop2016_df['STATE'] == 34) & (pop2016_df['COUNTY'].apply(lambda x: x in [5]))]
ny = pop2016_df[(pop2016_df['PRIMGEO_FLAG'] == 1) & (pop2016_df['STATE'] == 36) & (pop2016_df['COUNTY'].apply(lambda x: x in [119]))]

subdivs = subdivs.append(ca)
subdivs = subdivs.append(il)
subdivs = subdivs.append(nj)
subdivs = subdivs.append(ny)

subdivs['GEOID'] = 'fill this in'

subdivs.loc[(subdivs['SUMLEV'] == 157) & (subdivs['FUNCSTAT'] == 'A'), 'GEOID'] = subdivs['SUMLEV'].apply(lambda x: '{:03d}'.format(x)) + '00US' + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + subdivs['PLACE'].apply(lambda x: '{:05d}'.format(x))
subdivs.loc[(subdivs['SUMLEV'] == 157) & (subdivs['FUNCSTAT'] == 'F'), 'GEOID'] = subdivs['SUMLEV'].apply(lambda x: '{:03d}'.format(x)) + '00US' + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + subdivs['COUNTY'].apply(lambda x: '{:03d}'.format(x)) + 'xx'

#subdivs.loc[(subdivs['SUMLEV'] == 61), 'GEOID'] = subdivs['SUMLEV'].apply(lambda x: '{:02d}'.format(x)) + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + subdivs['COUNTY'].apply(lambda x: '{:03d}'.format(x)) + 'US' + subdivs['COUSUB'].apply(lambda x: '{:05d}'.format(x))
#subdivs.loc[(subdivs['SUMLEV'] == 71) & (subdivs['FUNCSTAT'] == 'A'), 'GEOID'] = subdivs['SUMLEV'].apply(lambda x: '{:02d}'.format(x)) + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + subdivs['COUNTY'].apply(lambda x: '{:03d}'.format(x)) + 'US' + subdivs['PLACE'].apply(lambda x: '{:05d}'.format(x))
#subdivs.loc[(subdivs['SUMLEV'] == 71) & (subdivs['FUNCSTAT'] == 'F'), 'GEOID'] = subdivs['SUMLEV'].apply(lambda x: '{:02d}'.format(x)) + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + subdivs['COUNTY'].apply(lambda x: '{:03d}'.format(x)) + 'US' + subdivs['COUSUB'].apply(lambda x: '{:05d}'.format(x))
#subdivs.loc[(subdivs['SUMLEV'] == 71) & (subdivs['FUNCSTAT'] == 'S'), 'GEOID'] = subdivs['SUMLEV'].apply(lambda x: '{:02d}'.format(x)) + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + subdivs['COUNTY'].apply(lambda x: '{:03d}'.format(x)) + 'US' + subdivs['PLACE'].apply(lambda x: '{:05d}'.format(x))

#subdivs.loc[(subdivs['STATE'] == 36), 'GEOID'] = subdivs['SUMLEV'].apply(lambda x: '{:02d}'.format(x)) + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + subdivs['COUNTY'].apply(lambda x: '{:03d}'.format(x)) + 'US' + subdivs['COUSUB'].apply(lambda x: '{:05d}'.format(x))

#subdivs.loc[((subdivs['SUMLEV'] == 61) | (subdivs['SUMLEV'] == 71)) & (subdivs['FUNCSTAT'] == 'A'), 'GEOID'] = '06171US' + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + subdivs['PLACE'].apply(lambda x: '{:05d}'.format(x))
#subdivs.loc[((subdivs['SUMLEV'] == 61) | (subdivs['SUMLEV'] == 71)) & (subdivs['FUNCSTAT'] != 'A'), 'GEOID'] = '06171US' + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + subdivs['PLACE'].apply(lambda x: '{:05d}'.format(x)) + 'FICT'


#subdivs.loc[subdivs['SUMLEV'] == 61, 'GEOID'] = subdivs['SUMLEV'].apply(lambda x: '{:02d}'.format(x)) + 'US' + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + subdivs['COUNTY'].apply(lambda x: '{:03d}'.format(x)) + subdivs['COUSUB'].apply(lambda x: '{:05d}'.format(x))

#subdivs.loc[(subdivs['SUMLEV'] == 71) & (subdivs['FUNCSTAT'] == 'A'), 'GEOID'] = subdivs['SUMLEV'].apply(lambda x: '{:03d}'.format(x)) + '00US' + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + subdivs['PLACE'].apply(lambda x: '{:05d}'.format(x))
#subdivs.loc[(subdivs['SUMLEV'] == 71) & (subdivs['FUNCSTAT'] == 'F'), 'GEOID'] = subdivs['SUMLEV'].apply(lambda x: '{:03d}'.format(x)) + 'xxUS' + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + subdivs['COUNTY'].apply(lambda x: '{:03d}'.format(x)) + subdivs['COUSUB'].apply(lambda x: '{:03d}'.format(x))
#subdivs.loc[(subdivs['SUMLEV'] == 71) & (subdivs['FUNCSTAT'] == 'S'), 'GEOID'] = subdivs['SUMLEV'].apply(lambda x: '{:03d}'.format(x)) + 'xxUS' + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + subdivs['COUNTY'].apply(lambda x: '{:03d}'.format(x)) + subdivs['COUSUB'].apply(lambda x: '{:03d}'.format(x))

#subdivs.loc[subdivs['FUNCSTAT'].apply(lambda x: x in ['F','S']), 'GEOID'] = 'FICT' + subdivs['GEOID']

#subdivs.loc[((subdivs['PLACE'] == 99990) & (subdivs['SUMLEV'] == 157)), 'GEOID'] = subdivs['SUMLEV'].apply(lambda x: '{:03d}'.format(x)) + 'US' + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + subdivs['COUNTY'].apply(lambda x: '{:03d}'.format(x)) + '9999'
#subdivs = subdivs.set_index('GEOID')

df = pandas.DataFrame()
df = df.append(counties)
df = df.append(subdivs)
df = df.set_index('GEOID', drop=False)

#newstates_df = pandas.read_csv('newstates.csv', index_col='GEOID')

#df = df.join(newstates_df)

#showcolumns = ['SUMLEV', 'STNAME', 'NAME', 'state_new', 'POPESTIMATE2016']
#showcolumns = ['SUMLEV', 'STNAME', 'COUNTY', 'NAME', 'PRIMGEO_FLAG', 'FUNCSTAT', 'POPESTIMATE2016']
showcolumns = ['SUMLEV', 'STNAME', 'COUNTY', 'NAME', 'PLACE', 'COUSUB', 'POPESTIMATE2016']

print('OK')

In [ ]:

#pop2016_df[showcolumns][(pop2016_df['STATE'] == 17) & (pop2016_df['COUNTY'] == 197) & (pop2016_df['SUMLEV'] == 50)]
# LA, CA: 10,137,915

# Burlington County, NJ: 449,284 - 71 does not add up!; 61: 40; 157: 10
# Westchester County, NY: 974,542 - 71 does not add up!; 61: 25; 157: 30

# Cook County, IL: 5,203,499 - 61: 31; 71: 243; 157: 138
# DuPage County, IL: 929,368 - 61: 10; 71: 79; 157: 40
# Kane County, IL: 531,715 - 61: 16; 71: 75; 157: 31
# Kendall County, IL: 124,695 - 61: 9; 71: 31; 157: 15
# Will County, IL: 689,529 - 71 does not add up!; 61: 24; 71: 99; 157: 40

#### DECISIONS
# California: use 157s
# New Jersey: use 61s
# New York: try both?
# Illinois: probably 61s, try 157s also?

pop2016_df[(pop2016_df['STATE'] == 36) & (pop2016_df['COUNTY'] == 119) & (pop2016_df['SUMLEV'] != 50)].groupby('SUMLEV')['POPESTIMATE2016'].sum()

#pop2016_df[showcolumns][(pop2016_df['STATE'] == 17) & (pop2016_df['COUNTY'] == 197) & (pop2016_df['SUMLEV'] != 50)].groupby('SUMLEV').size()
#SUMLEV
#61      31
#71     243
#157    138
#SUMLEV
#61     5203499
#71     5203499
#157    5203499

# Illinois (state 17): Cook (31), DuPage (43), Kane (89), Kendall (93), Will (197)

#pop2016_df[showcolumns][(pop2016_df['STATE'] == 6) & (pop2016_df['COUNTY'] == 37) & (pop2016_df['SUMLEV'] == 157)].groupby('PLACE').size()

In [ ]:
il61 = pop2016_df[(pop2016_df['SUMLEV'] == 61) & (pop2016_df['PRIMGEO_FLAG'] == 1) & (pop2016_df['STATE'] == 17) & (pop2016_df['COUNTY'].apply(lambda x: x in [31,43,89,93,197]))]
il71 = pop2016_df[(pop2016_df['SUMLEV'] == 71) & (pop2016_df['PRIMGEO_FLAG'] == 1) & (pop2016_df['STATE'] == 17) & (pop2016_df['COUNTY'].apply(lambda x: x in [31,43,89,93,197]))]
il157 = pop2016_df[(pop2016_df['SUMLEV'] == 157) & (pop2016_df['STATE'] == 17) & (pop2016_df['COUNTY'].apply(lambda x: x in [31,43,89,93,197]))]
#nj.to_csv('nj.csv')
il61.to_csv('il61.csv')
il71.to_csv('il71.csv')
il157.to_csv('il157.csv')

In [ ]:
ca157 = pop2016_df[(pop2016_df['SUMLEV'] == 157) & (pop2016_df['STATE'] == 6) & (pop2016_df['COUNTY'].apply(lambda x: x in [31,43,89,93,197]))]


In [ ]:
pop2016_df[pop2016_df['STATE'] == 6]